# Sentinel Data Generator - Log Analysis

This notebook analyzes the demo security data generated by the Sentinel Data Generator project.

**Tables Analyzed:**
- `SecurityEventDemo_CL` - Windows Security Events
- `CommonSecurityLog` - CEF/Syslog from security appliances
- `SigninLogDemo_CL` - Azure AD/Entra ID Sign-in Logs
- `SyslogDemo_CL` - Linux Syslog Events

**Requirements:** Run this notebook in Microsoft Sentinel with compute enabled.

## 1. Setup and Authentication

Connect to the Log Analytics workspace using Sentinel compute credentials.

In [ ]:
# Install required packages (if not already available)
# %pip install azure-monitor-query azure-identity pandas plotly kaleido

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Sentinel compute provides these automatically
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient, LogsQueryStatus

print("Libraries loaded successfully")

In [ ]:
# Configuration - Update with your workspace ID
# In Sentinel compute, this is often available via environment variables
import os

WORKSPACE_ID = os.environ.get("WORKSPACE_ID", "dec4f8ae-de22-4dff-b20c-0b3ac18c704f")
TIME_RANGE = timedelta(days=7)  # Query last 7 days

# Authenticate
credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

print(f"Connected to workspace: {WORKSPACE_ID}")

In [ ]:
def run_query(query: str, timespan: timedelta = TIME_RANGE) -> pd.DataFrame:
    """Execute a KQL query and return results as a DataFrame."""
    response = client.query_workspace(
        workspace_id=WORKSPACE_ID,
        query=query,
        timespan=timespan
    )
    
    if response.status == LogsQueryStatus.SUCCESS:
        table = response.tables[0]
        return pd.DataFrame(data=table.rows, columns=[col.name for col in table.columns])
    else:
        raise Exception(f"Query failed: {response.partial_error}")

print("Query helper function ready")

## 2. Overview - All Tables Summary

In [ ]:
# Get event counts from all 4 tables
overview_query = """
union 
  (SecurityEventDemo_CL | summarize Count=count() | extend Table="SecurityEventDemo_CL"),
  (SigninLogDemo_CL | summarize Count=count() | extend Table="SigninLogDemo_CL"),
  (SyslogDemo_CL | summarize Count=count() | extend Table="SyslogDemo_CL"),
  (CommonSecurityLog | summarize Count=count() | extend Table="CommonSecurityLog")
| project Table, Count
| order by Count desc
"""

df_overview = run_query(overview_query)
df_overview

In [ ]:
# Visualize table distribution
fig = px.pie(
    df_overview, 
    values='Count', 
    names='Table',
    title='Event Distribution by Table',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.show()

## 3. SecurityEventDemo_CL Analysis

Windows Security Events including failed/successful logons, privilege escalation, and account management.

In [ ]:
# Security events by EventID
security_events_query = """
SecurityEventDemo_CL
| summarize Count=count() by EventID, Activity
| order by Count desc
"""

df_security = run_query(security_events_query)
df_security

In [ ]:
# Security events over time
security_timeline_query = """
SecurityEventDemo_CL
| summarize Count=count() by bin(TimeGenerated, 1h), EventID
| order by TimeGenerated asc
"""

df_security_time = run_query(security_timeline_query)

fig = px.bar(
    df_security, 
    x='EventID', 
    y='Count',
    color='Activity',
    title='Windows Security Events by Event ID',
    text='Count'
)
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
# Failed logins analysis (brute force detection)
failed_logins_query = """
SecurityEventDemo_CL
| where EventID == 4625
| summarize FailedAttempts=count() by IpAddress, Account, Computer
| where FailedAttempts >= 5
| order by FailedAttempts desc
| take 20
"""

df_failed = run_query(failed_logins_query)
print("Potential Brute Force Sources (5+ failed logins):")
df_failed

In [ ]:
if not df_failed.empty:
    fig = px.bar(
        df_failed.head(10), 
        x='IpAddress', 
        y='FailedAttempts',
        color='Account',
        title='Top Failed Login Sources (Potential Brute Force)',
        text='FailedAttempts'
    )
    fig.show()

## 4. CommonSecurityLog Analysis

CEF events from security appliances including firewall, IDS, and threat intelligence.

In [ ]:
# Events by vendor and event class
cef_vendor_query = """
CommonSecurityLog
| summarize Count=count() by DeviceVendor, DeviceEventClassID
| order by Count desc
| take 20
"""

df_cef = run_query(cef_vendor_query)
df_cef

In [ ]:
fig = px.treemap(
    df_cef, 
    path=['DeviceVendor', 'DeviceEventClassID'], 
    values='Count',
    title='CommonSecurityLog Events by Vendor and Event Type',
    color='Count',
    color_continuous_scale='RdYlGn_r'
)
fig.show()

In [ ]:
# Threat intelligence and intrusion detections
threats_query = """
CommonSecurityLog
| where DeviceEventClassID in ("threat:match", "intrusion:detected", "malware:detected")
| summarize Count=count() by DeviceVendor, DeviceEventClassID, SourceIP
| order by Count desc
| take 15
"""

df_threats = run_query(threats_query)
print("Security Threats Detected:")
df_threats

In [ ]:
# Firewall denies by source IP
firewall_query = """
CommonSecurityLog
| where DeviceEventClassID == "traffic:deny"
| summarize DenyCount=count() by SourceIP, DestinationIP
| where DenyCount >= 5
| order by DenyCount desc
| take 15
"""

df_firewall = run_query(firewall_query)
print("Firewall Blocks (5+ denies):")
df_firewall

## 5. SigninLogDemo_CL Analysis

Azure AD/Entra ID sign-in events including brute force and credential stuffing attacks.

In [ ]:
# Sign-in results overview
signin_results_query = """
SigninLogDemo_CL
| summarize Count=count() by ResultType, ResultDescription
| order by Count desc
"""

df_signin = run_query(signin_results_query)
df_signin

In [ ]:
# Visualize sign-in results
fig = px.bar(
    df_signin, 
    x='ResultDescription', 
    y='Count',
    color='ResultType',
    title='Azure AD Sign-in Results',
    text='Count'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
# Risky sign-ins
risky_signin_query = """
SigninLogDemo_CL
| where RiskLevelDuringSignIn in ("medium", "high")
| summarize Count=count() by UserPrincipalName, IPAddress, Location, RiskLevelDuringSignIn
| order by Count desc
| take 15
"""

df_risky = run_query(risky_signin_query)
print("Risky Sign-ins Detected:")
df_risky

In [ ]:
# Sign-ins by location
signin_location_query = """
SigninLogDemo_CL
| summarize Count=count(), 
            Success=countif(ResultType == "0"), 
            Failed=countif(ResultType != "0") 
  by Location
| extend FailureRate = round(100.0 * Failed / Count, 1)
| order by Count desc
"""

df_location = run_query(signin_location_query)
df_location

In [ ]:
fig = px.bar(
    df_location, 
    x='Location', 
    y=['Success', 'Failed'],
    title='Sign-in Attempts by Location',
    barmode='stack',
    color_discrete_map={'Success': 'green', 'Failed': 'red'}
)
fig.show()

## 6. SyslogDemo_CL Analysis

Linux syslog events including SSH authentication, sudo abuse, and service failures.

In [ ]:
# Syslog by facility and severity
syslog_overview_query = """
SyslogDemo_CL
| summarize Count=count() by Facility, SeverityLevel
| order by Count desc
"""

df_syslog = run_query(syslog_overview_query)
df_syslog

In [ ]:
fig = px.sunburst(
    df_syslog, 
    path=['Facility', 'SeverityLevel'], 
    values='Count',
    title='Syslog Events by Facility and Severity',
    color='SeverityLevel',
    color_discrete_map={
        'emerg': '#8B0000', 'alert': '#FF0000', 'crit': '#FF4500',
        'err': '#FF6347', 'warning': '#FFA500', 'notice': '#FFD700',
        'info': '#90EE90', 'debug': '#ADD8E6'
    }
)
fig.show()

In [ ]:
# SSH authentication failures
ssh_failures_query = """
SyslogDemo_CL
| where ProcessName == "sshd" and SyslogMessage has "Failed"
| summarize FailedAttempts=count() by Computer, HostIP
| where FailedAttempts >= 3
| order by FailedAttempts desc
| take 15
"""

df_ssh = run_query(ssh_failures_query)
print("SSH Brute Force Targets (3+ failures):")
df_ssh

In [ ]:
# Service failures
service_failures_query = """
SyslogDemo_CL
| where Facility == "daemon" and SeverityLevel in ("err", "crit")
| summarize Count=count() by Computer, ProcessName, SeverityLevel
| order by Count desc
| take 15
"""

df_services = run_query(service_failures_query)
print("Service Failures:")
df_services

## 7. Security Summary Dashboard

In [ ]:
# Create a comprehensive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Events by Table', 
        'Windows Security Events',
        'Sign-in Results', 
        'Syslog Severity'
    ),
    specs=[
        [{"type": "pie"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ]
)

# Plot 1: Events by table
fig.add_trace(
    go.Pie(labels=df_overview['Table'], values=df_overview['Count'], name="Tables"),
    row=1, col=1
)

# Plot 2: Security Events
fig.add_trace(
    go.Bar(x=df_security['EventID'].astype(str), y=df_security['Count'], name="Security Events"),
    row=1, col=2
)

# Plot 3: Sign-in Results
fig.add_trace(
    go.Bar(x=df_signin['ResultDescription'], y=df_signin['Count'], name="Sign-ins"),
    row=2, col=1
)

# Plot 4: Syslog Severity
syslog_severity = df_syslog.groupby('SeverityLevel')['Count'].sum().reset_index()
fig.add_trace(
    go.Bar(x=syslog_severity['SeverityLevel'], y=syslog_severity['Count'], name="Syslog"),
    row=2, col=2
)

fig.update_layout(
    height=800, 
    title_text="Sentinel Data Generator - Security Overview Dashboard",
    showlegend=False
)
fig.show()

## 8. Key Findings Summary

In [ ]:
# Generate summary statistics
total_events = df_overview['Count'].sum()
failed_logins_count = df_security[df_security['EventID'] == 4625]['Count'].sum() if not df_security.empty else 0
success_signins = df_signin[df_signin['ResultType'] == '0']['Count'].sum() if not df_signin.empty else 0
failed_signins = df_signin[df_signin['ResultType'] != '0']['Count'].sum() if not df_signin.empty else 0

print("="*60)
print("SECURITY ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal Events Analyzed: {total_events:,}")
print(f"\nWindows Security:")
print(f"  - Failed Logons (4625): {failed_logins_count:,}")
print(f"  - Potential Brute Force Sources: {len(df_failed)}")
print(f"\nAzure AD Sign-ins:")
print(f"  - Successful: {success_signins:,}")
print(f"  - Failed: {failed_signins:,}")
print(f"  - Risky Sign-ins: {len(df_risky)}")
print(f"\nNetwork Security (CommonSecurityLog):")
print(f"  - Threat Detections: {len(df_threats)}")
print(f"  - Firewall Blocks: {len(df_firewall)}")
print(f"\nLinux Systems (Syslog):")
print(f"  - SSH Attack Targets: {len(df_ssh)}")
print(f"  - Service Failures: {len(df_services)}")
print("\n" + "="*60)